## 03: Generate counts

This script takes a directory of `.csv` files containing entity counts by month in the following format:

```csv
,2012-01,2012-02
meat,1011.0,873.0
salt,805.0,897.0
chicken,694.0,713.0
```

It sums the counts from all files, only keeps the `N` most common records and calculates the variance, scaled by the average. This helps select a more "interesting" subset of entities with the most variance over time. The result are the most variant entities (minus the most frequent, which tend to be less interesting). The result can be used to create an interactive [bar chart race visualization](https://public.flourish.studio/visualisation/1532208/). 

In [14]:
INPUT_DIR = "./counts"               # directory of counts file(s) created in the previous step
OUTPUT_FILE = "./output_counts.csv"  # path to output file
MOST_COMMON = 10_000                 # number of most common entities to keep
DROP_MOST_FREQUENT = 10              # number of most frequent entities to drop
N_TOTAL = 50                         # number of results to export

In [ ]:
!pip install pandas

In [15]:
import csv
from collections import Counter, defaultdict
from pathlib import Path
import pandas as pd

In [16]:
def read_csv(file_):
    counts = Counter()
    for row in csv.DictReader(file_):
        term = row[""]
        for year, freq in row.items():
            if year != "" and freq:
                counts[(term, year)] = int(float(freq))
    return counts


def prune_rows(counts_by_term, n):
    totals = Counter()
    for term, counts in counts_by_term.items():
        if "Total" in counts:
            total = counts["Total"]
        else:
            total = sum(counts.values())
        totals[term] = total
    pruned = defaultdict(dict)
    for term, _ in totals.most_common(n):
        pruned[term] = counts_by_term[term]
    return pruned


def sum_counts(directory, n=10000):
    directory = Path(directory)
    counts = Counter()
    for path in directory.glob("**/*.csv"):
        with path.open("r", encoding="utf8") as file_:
            counts.update(read_csv(file_))
    by_term = defaultdict(Counter)
    for (term, month), freq in counts.items():
        by_term[term][month] = freq
    records = prune_rows(by_term, n)
    months = set()
    for term, counts in records.items():
        months.update(counts.keys())
    fields = ["Term"] + list(sorted(months))
    rows = []
    for term, month_freqs in records.items():
        month_freqs["Term"] = term
        for month in months:
            month_freqs.setdefault(month, 0.0)
        rows.append(month_freqs)
    return pd.DataFrame.from_records(rows, index="Term", columns=fields)


def sort_by_frequency(df):
    most_common = df.sum(axis=1)
    most_common.sort_values(ascending=False, inplace=True)
    return df.loc[most_common.index]


def drop_most_frequent(df, n):
    return sort_by_frequency(df)[n:]


def get_most_variant(df, n, mean_weight=False):
    cvars = df.var(axis=1)
    if mean_weight:
        cvars = cvars / df.mean(axis=1)
    cvars = cvars.sort_values(ascending=False)
    return df.loc[cvars.index][:n]

In [18]:
DF = sum_counts(INPUT_DIR, MOST_COMMON)
DF

,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
Term,,,,,,,,,,,,,,,,,,,,,
salt,805,897,271,664,690,787,718,661,805,1165,...,2202,2209,2456,3093,2606,2460,2441,2798,2061,3372
meat,1011,873,305,614,771,655,704,834,655,1020,...,1825,1746,2427,2942,2109,2025,2437,2234,1936,3379
chicken,694,713,211,617,534,496,703,623,731,875,...,1768,1508,1840,2595,2057,2201,2023,2331,2069,3213
garlic,590,679,180,635,501,587,560,600,634,623,...,1369,1577,1895,2585,1949,1830,1816,2137,1518,2510
rice,805,542,219,403,357,467,376,435,367,585,...,2295,1520,1262,2754,2302,1699,2011,2189,1450,2089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ricers,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
wild hog,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
cavendish bananas,0,0,0,0,0,0,0,0,0,0,...,0,2,1,0,0,0,0,0,0,0


In [19]:
SUBSET = drop_most_frequent(DF, DROP_MOST_FREQUENT)
SUBSET = get_most_variant(SUBSET, N_TOTAL, mean_weight=True)[:200]
SUBSET = sort_by_frequency(SUBSET)
SUBSET = SUBSET.cumsum(axis=1)
SUBSET

,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
Term,,,,,,,,,,,,,,,,,,,,,
potatoes,343,654,773,963,1194,1402,1610,1905,2168,2568,...,37277,38479,39734,41033,42277,43420,44668,45723,46768,48460
onions,330,657,796,1076,1540,1850,2201,2510,2829,3176,...,35786,36703,37557,39298,40462,41567,42691,43967,44759,46478
bacon,439,799,983,1325,1818,2237,2766,3109,3495,3914,...,35680,36357,37514,38969,39708,40429,41128,42064,42798,43857
pasta,451,679,763,1054,1361,1648,2016,2284,2534,3058,...,33008,34048,35010,36387,37512,38363,39412,40591,41829,43308
beef,285,631,765,1089,1314,1550,1785,2056,2258,2638,...,30900,31667,32631,33804,34735,35566,36499,37372,38180,39423
egg,302,566,700,914,1261,1500,1777,2036,2334,2614,...,30880,31433,32313,33304,34125,34921,35955,36997,37758,39253
olive oil,348,698,796,1108,1433,1759,2097,2399,2713,3045,...,31935,32580,33199,34114,35131,35882,36704,37486,38133,39157
onion,307,622,718,957,1296,1589,1862,2130,2433,2818,...,31212,32048,32832,34037,34811,35594,36292,37255,37930,39118
steak,357,785,915,1235,1485,1701,1967,2339,2591,2935,...,31193,31671,32579,33593,34361,35021,35962,36693,37490,38871


In [ ]:
SUBSET.to_csv(OUTPUT_FILE)